In [2]:
%matplotlib inline

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

In [3]:
import pickle

with open('my_dataset.pickle', 'rb') as data:
    dataset = pickle.load(data)

### <ins>Point 1.</ins>

### <ins>Should we take only stat feats or all of them?.</ins>

In [ ]:
X, X_test, y, y_test = train_test_split(
    df[features].values, # X  come numpy array
    df['label'].values, # y come numpy array
    stratify = df['label'], # stratify the dataset based on class labels
    train_size = 0.7, # percentage of training set
    random_state = 15 #for random chose of samples
)

In [ ]:
X_train, X_validation, y_train, y_validation = train_test_split(
    df[features].values, # X  come numpy array
    df['label'].values, # y come numpy array
    stratify = df['label'], # stratify the dataset based on class labels
    train_size = 0.7143, # percentage of training set
    random_state = 15 #for random chose of samples
)

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_validation = scaler.transform(X_validation)
X_test = scaler.transform(X_test)

### <ins>Point 2.</ins>

In [ ]:
import random

y_lossy = y_train[y_train==1].index
y_lossless = y_train[y_train==0].index

num_of_el_to_pick = len(y_lossy)

index_lossy = random.sample(sorted(set(y_lossy)),num_of_el_to_pick)
index_lossless = random.sample(sorted(set(y_lossless)),num_of_el_to_pick)

df_lossless = X_train.loc[index_lossless]
df_lossy = X_train.loc[index_lossy]

X_balanced = pd.concat([df_lossless, df_lossy])
Y_balanced = pd.concat([y_train.loc[index_lossless], y_train.loc[index_lossy]])


gnb = GaussianNB()
gnb.fit(X_balanced, Y_balanced)
y_test_pred = gnb.predict(X_test)

print(classification_report(y_test, y_test_pred))

### <ins>Point 3.</ins>

In [ ]:
m1 = KNeighborsClassifier(n_neighbors=3)
m1.fit(X_train_np, y_train)
y_test_pred1 = m1.predict(X_test_np)
y_train_pred1 = m1.predict(X_train_np)

In [ ]:
m1 = DecisionTreeRegressor(random_state=0)
m1.fit(X_train_np, y_train)
y_test_pred1 = m1.predict(X_test_np)
y_train_pred1 = m1.predict(X_train_np)

In [ ]:
m3 = RandomForestClassifier(n_estimators=30)
m3.fit(X_train_np, y_train)
y_test_pred3 = m3.predict(X_test_np)
y_train_pred3 = m3.predict(X_train_np)